In [9]:
%%writefile hwaes.py

#%reload_ext autoreload
#%autoreload 2
import os
import time
import numpy as np
from random import randrange
import time


from Crypto.Cipher import AES

np.set_printoptions(formatter={'int':hex})

from PyPci import pci
dev = pci.pci()
def task_writeDIO(addr, data):
    dev.write(data, addr)
    
def task_readDIO(addr, length):
    data2 = dev.read(addr, length)
    #for i in range (length):
    #    print hex(i), "\t:", hex(data2[i])
    return data2
        
def task_writeDMA(data):
    dev.writeDMA(data)
    return 0
    
def task_readDMA(length):
    data3 = dev.readDMA(length)
    return data3

PAIR = 0
BURST_PAIRS = 1
BURST_SINGLEKEY = 2
SEARCH_KEY = 3
BURST_OLDKEY = 4
IDENTIFIER = 0xAE
RETIDENTIFIER = 0xEA

# HEADER
START_BYTE = 0xAB

# Command
IDLE =  0x00
SEND =  0x01
RECIEVE =  0x04
RESET = 0x02
BUSY =  0x03

def test_pair_packet():    
    key0 = [0x0706050403020100,
            0x0f0e0d0c0b0a0908,
            0x1716151413121110,
            0x1f1e1d1c1b1a1918]
    data0 = [0x7766554433221100,
             0xffeeddccbbaa9988]
    key1 = [0x0706050403020100,
            0x0f0e0d0c0b0a0908,
            0x1716151413121110,
            0x1f1e1d1c1b1a1918]
    data1 = [0x7766554433221100,
             0xffeeddccbbaa9988]
    nk = 0x8
    enc = 0x1
    mode = PAIR
    size = 0
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    databuff.append(head)
    for i in key0:
        databuff.append(i)
    for i in data0:
        databuff.append(i)
    
    databuff.append(0x0)
    databuff.append(0x0)
    databuff.append(0x0)
    databuff.append(0x0)
    databuff.append(0x0)
    databuff.append(0x0)
    databuff.append(0x0)
    databuff.append(0x0)
    return databuff

def test_burst_pairs_packet():
    
    key0 = [0x0706050403020100,
            0x0f0e0d0c0b0a0908,
            0x1716151413121110,
            0x1f1e1d1c1b1a1918]
    data0 = [0x7766554433221100,
             0xffeeddccbbaa9988]
    key1 = [0x0206050403020100,
            0x020e0d0c0b0a0908,
            0x1216151413121110,
            0x121e1d1c1b1a1918]
    data1 = [0x7266554433221100,
             0xf2eeddccbbaa9988]
    nk = 0x8
    enc = 0x1
    mode = BURST_PAIRS
    size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    databuff.append(head)
    for j in range(size/2):
        for i in key0:
            databuff.append(i)
        for i in data0:
            databuff.append(i)
        for i in key1:
            databuff.append(i)
        for i in data1:
            databuff.append(i)
    return databuff
        

def test_burst_singlekey_packet():    
    key = [0x0706050403020100,
           0x0f0e0d0c0b0a0908,
           0x1716151413121110,
           0x1f1e1d1c1b1a1918]
    data = [0x7766554433221100,
            0xffeeddccbbaa9988]
    nk = 0x8
    enc = 0x1
    mode = BURST_SINGLEKEY
    size = 0x10
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    databuff.append(head)
    for i in key:
        databuff.append(i)
    for j in range(size):
        for i in data:
            databuff.append(i)
    return databuff

def test_search_key_packet():    
    key = [0x0706050403120100,
           0x0f0e0d0c0b0a0908,
           0x1716151413121110,
           0x1f1e1d1c1b1a1918]
    data = [0x7766554433221100,
            0xffeeddccbbaa9988]
    result = [0xbf456751cab7a28e,
              0x8960494b9049fcea]       
    nk = 0x8
    enc = 0x1
    mode = SEARCH_KEY
    searchspace = 16  # 0 to (64-20)
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ searchspace << 24 ^ packetid << 48
    databuff = []
    databuff.append(head)
    for i in key:
        databuff.append(i)
    for i in data:
        databuff.append(i)
    for i in result:
        databuff.append(i)
    return databuff

def encapsulate_packet(source, sink, packet):
    rambuff = []
    command = makeheader(SEND, len(packet)+1, 2, len(packet)+2)
    rambuff.append(0)
    rambuff.append(0)
    rambuff.append(0xFF ^ sink << 8 ^ source << 24 ^ len(packet) << 40)
    for i in packet:
        rambuff.append(i)
    #print rambuff
    return command, rambuff
'''
    rambuff = []
    rambuff.append(makeheader(SEND, len(packet)+1, 2, len(packet)+2))
    rambuff.append(0)
    rambuff.append(0xFF ^ sink << 8 ^ source << 24 ^ len(packet) << 40)
    for i in packet:
        rambuff.append(i)
    #print rambuff
    return rambuff'''

def makeheader(cmdstat, length, valid_start, valid_stop):
    CMDSTAT_PTR = 1
    LENGTH_PTR = 2
    VALID_START_PTR = 4
    VALID_STOP_PTR = 6
    header = (START_BYTE | (cmdstat << CMDSTAT_PTR*8) | (length << LENGTH_PTR*8) | (valid_start << VALID_START_PTR*8) | 
              (valid_stop << VALID_STOP_PTR*8))
    return header

def reset(nodeptr = 0xFFFF):
    rst_packet = [0, 0, 0x0000010000FFFFFF, 0xFFFFFFFFFFFFFFFF]
    rst_packet[2] = rst_packet[2] & (nodeptr << 8)
    command = makeheader(SEND, 2, 2, 3)
            
    data = np.array(rst_packet, dtype=np.uint64)
    command = np.array([command, 0], dtype=np.uint64)
    
    task_writeDMA(data)
    task_writeDMA(command)
    task_writeDMA(null)
    fifo_status = np.array([0, 0x00000000000004AB], dtype=np.uint64)
    task_writeDMA(fifo_status)
        
def setnode(nodeptr = 0xFFFF, new_nodeptr= 0xFFFF):
    packet = [0, 0, 0x10000FFFFFF, 0x00000000FFFFAADD]
    packet[2] = packet[2] & (nodeptr << 8)
    packet[3] = packet[3] & (new_nodeptr << 16)
    
    command = makeheader(SEND, 2, 2, 3)            
    data = np.array(packet, dtype=np.uint64)
    
    command = np.array([command, 0], dtype=np.uint64)
    
    task_writeDMA(data)
    task_writeDMA(command)
    task_writeDMA(null)
    fifo_status = np.array([0, 0x00000000000004AB], dtype=np.uint64)
    task_writeDMA(fifo_status)

def burst_pairs_test(size = 8):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = BURST_PAIRS
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    cryptbuff = []
    databuff.append(head)
    for j in range(size):
        key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
        #keybuffer = np.getbuffer(key)
        data = np.array([randrange(2**base) for i in range (datasize/base)], dtype=np.uint64)
        #databuffer = np.getbuffer(data)
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        encryptedbuffer = aes.encrypt(data)
        encrypted = np.frombuffer(encryptedbuffer, dtype=np.uint64)
        
        for i in key:
            databuff.append(i)
        for i in data:
            databuff.append(i)
        for i in encrypted:
            cryptbuff.append(i)
            
    command, data = encapsulate_packet(0, 1, databuff)
    data = np.array(data, dtype=np.uint64)
    command = np.array([command, 0], dtype=np.uint64)
    cryptbuff = np.array(cryptbuff, dtype=np.uint64)
    #print data , command
    task_writeDMA(data)
    task_writeDMA(command)
    task_writeDMA(null)
    fifo_status = np.array([0, 0x00000000000004AB], dtype=np.uint64)
    task_writeDMA(fifo_status)
    header = task_readDIO(3, 1)
    if int(header[0]) & 0xFF == 0xea:
        if packetid == int(header[0]) >> 48:
            print "packet recieved"
            cryptbuff1 = task_readDMA(size * 2 + 4)
            #print cryptbuff1
            if (cryptbuff1[4:] == cryptbuff).all():
                print "encryption is correct"
            else:
                print "encryption is incorrect"
        else:
            print "wrong/no packet recieved"
    cryptbuff1 = task_readDMA(size * 2 + 4)
    #print cryptbuff1 , "##"
    #print cryptbuff
    
def burst_singlekey_test(size = 0x10):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = BURST_SINGLEKEY
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    cryptbuff = []
    databuff.append(head)
    key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
    #keybuffer = np.getbuffer(key)
    for i in key:
        databuff.append(i)
    #print key
    for j in range(size):
        data = np.array([randrange(2**base) for i in range (datasize/base)], dtype=np.uint64)
        #databuffer = np.getbuffer(data)
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        encryptedbuffer = aes.encrypt(data)
        encrypted = np.frombuffer(encryptedbuffer, dtype=np.uint64)
        
        for i in data:
            databuff.append(i)
        for i in encrypted:
            cryptbuff.append(i)
        
        #print '$$', data
            
    command, data = encapsulate_packet(0, 1, databuff)
    data = np.array(data, dtype=np.uint64)
    command = np.array([command, 0], dtype=np.uint64)
    cryptbuff = np.array(cryptbuff, dtype=np.uint64)
    #print data , command
    task_writeDMA(data)
    task_writeDMA(command)
    task_writeDMA(null)
    fifo_status = np.array([0, 0x00000000000004AB], dtype=np.uint64)
    task_writeDMA(fifo_status)
    header = task_readDIO(3, 1)
    if int(header[0]) & 0xFF == 0xea:
        if packetid == int(header[0]) >> 48:
            print "packet recieved"
            cryptbuff1 = task_readDMA(size * 2 + 4)
            #print cryptbuff1
            if (cryptbuff1[4:] == cryptbuff).all():
                print "encryption is correct"
    
    cryptbuff1 = task_readDMA(size * 2 + 4)
    #print cryptbuff1 , "##"
    #print cryptbuff
    
def search_key(key, data, crypto, searchspace = 0, core = 2):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = SEARCH_KEY
    null = np.array([0x00, 0x00], dtype=np.uint64)
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ searchspace << 24 ^ packetid << 48
    databuff = []
    #cryptbuff = []
    databuff.append(head)
    #key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
    #keybuffer = np.getbuffer(key)
    #key[0] = 0xFFFFFFFFFFFFFF00
    #print '#', key
    data = np.frombuffer(data, dtype=np.uint64) 
    #np.array([randrange(2**base) for i in range (datasize/base)], dtype=np.uint64)
    #databuffer = np.getbuffer(data)

    mode = AES.MODE_ECB
    #aes = AES.new(key, mode)
    #encryptedbuffer = aes.encrypt(data)
    encrypted = np.frombuffer(crypto, dtype=np.uint64)
        
    #key[0] = key[0] & 0xFFFFFFFFFFF00000
    for i in key:
        databuff.append(i)
    for i in data:
        databuff.append(i)
    for i in encrypted:
        databuff.append(i)
            
    command, data = encapsulate_packet(0, core, databuff)
    data = np.array(data, dtype=np.uint64)
    command = np.array([command, 0], dtype=np.uint64)
    #cryptbuff = np.array(cryptbuff, dtype=np.uint64)
    #print data, command
    task_writeDMA(data)
    task_writeDMA(command)
    task_writeDMA(null)
    fifo_status = np.array([0, 0x00000000000000AB], dtype=np.uint64)
    task_writeDMA(fifo_status)
    #time.sleep(1)
    #for i in range(100000):
    header = task_readDIO(3, 1)
    if int(header[0]) & 0xFF == 0xea:
        if packetid == int(header[0]) >> 48:
            #print "packet recieved"
            cryptbuff1 = task_readDMA(13)
            #print hex(cryptbuff1[5] - 0x1e)
            return int(cryptbuff1[5] - 0x1e), packetid
                #print cryptbuff1 
                #print 'Count #', i
                #break
                
    return 0, packetid

def check_for_search_packet(packetid):
    header = task_readDIO(3, 1)
    if int(header[0]) & 0xFF == 0xea:
        if packetid == int(header[0]) >> 48:
            cryptbuff1 = task_readDMA(13)
            return int(cryptbuff1[5] - 0x1e), packetid
    return 0, 0

PCI Initialized
BAR0 Size: 0x10000
BAR1 Size: 0x100000


In [10]:
command, data = encapsulate_packet(0, 0, test_pair_packet())
data = np.array(data, dtype=np.uint64)
command = np.array([command], dtype=np.uint64)

command, data = encapsulate_packet(0, 2, test_burst_pairs_packet())
data = np.array(data, dtype=np.uint64)
command = np.array([command], dtype=np.uint64)

command, data = encapsulate_packet(0, 1, test_burst_singlekey_packet())
data = np.array(data, dtype=np.uint64)
command = np.array([command], dtype=np.uint64)

command, data = encapsulate_packet(0, 2, test_search_key_packet())
data = np.array(data, dtype=np.uint64)
command = np.array([command], dtype=np.uint64)

null = np.array([0x00, 0x00], dtype=np.uint64)

In [11]:
def burst_pairs_test(nodeptr=0xFF, size = 8):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = BURST_PAIRS
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    cryptbuff = []
    databuff.append(head)
    for j in range(size):
        key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
        #keybuffer = np.getbuffer(key)
        data = np.array([randrange(2**base) for i in range (datasize/base)], dtype=np.uint64)
        #databuffer = np.getbuffer(data)
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        encryptedbuffer = aes.encrypt(data)
        encrypted = np.frombuffer(encryptedbuffer, dtype=np.uint64)
        
        for i in key:
            databuff.append(i)
        for i in data:
            databuff.append(i)
        for i in encrypted:
            cryptbuff.append(i)
    
    '''databuff = [0xea1e0000080181aeL, 0xce7004b16b682f1fL,
                0x418b836ee324af00L, 0xbe5d12d613ed66dfL, 0x5d2057e02d67c7dL,
                0x6d7a0686f2ab9f97L, 0x6b6ed8b709fe8bc7L, 0xd5bdae838d8b1efdL,
                0x15983a69d8d660d7L, 0x72fb677efdf1d219L, 0x32042adc11822b7bL,
                0xcc4b4cf3b6288b9aL, 0x22794adbb5a09e4dL, 0x66dcc10593f7baa1L,
                0xa724667d83deb3aL, 0x8c4a1e50d75cd8b1L, 0x798c53bec5dc751dL,
                0x9e470370c1aa44d1L, 0xc65de063d87c8260L, 0x137030b8535e83feL,
                0xf92e0539299e0173L, 0xb1cd98b7af7ced9bL, 0xbc55cf233074ca75L,
                0x996310a612a58e3dL, 0x84e03d772b9cd878L, 0xa6b4485179678149L,
                0xae64b9d1b53c0b40L, 0x7a0e1f616405feb4L, 0xc0a19ee57faffe44L,
                0xeac381050be5aceL, 0xda2c57a019a8a4bfL, 0x31380f366e729c3eL,
                0xf3e53ee059b05825L, 0xc7bdf9cac2be80acL, 0x68e16115e969e947L,
                0xbd03c88898ada873L, 0xf460c6ba305cfd7eL, 0x680fad56b9e6e760L,
                0x5d5a58de601f20e1L, 0x8587c7052142c47bL, 0x4541aa4a999eb3f0L,
                0xff8b7c4913488176L, 0x2bc1db551c97622eL, 0x737405dce95aa6bbL,
                0xcd966d5be9f85618L, 0x209a57031980f762L, 0x34ac73a8fc38cd56L,
                0x9c94f6a3b236504L, 0x8197405085f3b42fL]'''
    command, data = encapsulate_packet(0, nodeptr, databuff)
    data = np.array(data, dtype=np.uint64)
    command = np.array([command, 0], dtype=np.uint64)
    cryptbuff = np.array(cryptbuff, dtype=np.uint64)
    print data , command
    task_writeDMA(data)
    task_writeDMA(command)
    task_writeDMA(null)
    fifo_status = np.array([0, 0x00000000000004AB], dtype=np.uint64)
    task_writeDMA(fifo_status)
    for i in range(200):
        print i,
        header = task_readDIO(3, 1)
        if int(header[0]) & 0xFF == 0xea:
            if packetid == int(header[0]) >> 48:
                print "packet recieved"
                print i
                cryptbuff1 = task_readDMA(size * 2 + 4)
                print cryptbuff1
                if (cryptbuff1[4:] == cryptbuff).all():
                    print "encryption is correct"
                else:
                    print "encryption is incorrect"
                break
            #else:
            #    print "wrong/no packet recieved"
        cryptbuff1 = task_readDMA(size * 2 + 4)
    #print cryptbuff1 , "##"
    #print cryptbuff

In [12]:
def burst_singlekey_test(nodeptr = 0xFF, size = 0x10):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = BURST_SINGLEKEY
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    cryptbuff = []
    databuff.append(head)
    key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
    #keybuffer = np.getbuffer(key)
    for i in key:
        databuff.append(i)
    #print key
    for j in range(size):
        data = np.array([randrange(2**base) for i in range (datasize/base)], dtype=np.uint64)
        #databuffer = np.getbuffer(data)
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        encryptedbuffer = aes.encrypt(data)
        encrypted = np.frombuffer(encryptedbuffer, dtype=np.uint64)
        
        for i in data:
            databuff.append(i)
        for i in encrypted:
            cryptbuff.append(i)
        
        #print '$$', data
            
    command, data = encapsulate_packet(0, nodeptr, databuff)
    data = np.array(data, dtype=np.uint64)
    command = np.array([command, 0], dtype=np.uint64)
    cryptbuff = np.array(cryptbuff, dtype=np.uint64)
    print data , command
    task_writeDMA(data)
    task_writeDMA(command)
    task_writeDMA(null)
    fifo_status = np.array([0, 0x00000000000004AB], dtype=np.uint64)
    task_writeDMA(fifo_status)
    header = task_readDIO(3, 1)
    if int(header[0]) & 0xFF == 0xea:
        if packetid == int(header[0]) >> 48:
            print "packet recieved"
            cryptbuff1 = task_readDMA(size * 2 + 4)
            #print cryptbuff1
            if (cryptbuff1[4:] == cryptbuff).all():
                print "encryption is correct"
    
    cryptbuff1 = task_readDMA(size * 2 + 4)
    #print cryptbuff1 , "##"
    #print cryptbuff

In [13]:
reset(nodeptr = 0xFFFF)
setnode(nodeptr = 0xFFFF, new_nodeptr= 0xFFFF)

In [15]:
burst_pairs_test(nodeptr = 0xFFFF, size = 8)
#burst_pairs_test(size = 8)
#burst_pairs_test(size = 8)
#burst_pairs_test(size = 8)
#burst_pairs_test(size = 8)

[0x0L 0x0L 0x310000ffffffL 0xb90e0000080181aeL 0x5cc0023a3288f058L
 0xeb85a7daf74b6ccbL 0x71ab98fb049ab1c7L 0xaa46848f88afdbc8L
 0x5fa5df49df4ef86L 0x5d620c00abc676f4L 0xd815dbc2738d6aafL
 0x8c8456b7d8c083f0L 0x1b29457ac1419161L 0xe6bfa7fa919620fL
 0x26e7d454ff9e7370L 0x266ed1741ada3084L 0x8798e80105470e5cL
 0x5026eb33ece4d18L 0x69969f449a1687fbL 0x62b8f9d3b53df24fL
 0x8ca6d342c518f3dfL 0xfe5bdf367cddd327L 0x94f56936917d5af6L
 0x5a6284f30aeb4637L 0x6a7763fb282f22cbL 0x61b0634aad824e2eL
 0x4cf01a2b54ac0fddL 0x6eee2ef604fbeea8L 0xb0fc69e52d688cf6L
 0xedab60b6a226bbb0L 0x8d51f05b3fb81adfL 0x29c5a6cfe5ae416aL
 0x327bcc11bfb4b9d1L 0xaf7872a613aafb6cL 0xe96b2a6f7609cc95L
 0x326e3bfce06184e0L 0x913fac9b69e90c28L 0xfe07d4d5cf73bb93L
 0x1a9dd7c25b6123f2L 0x8cdb9a20e06e063eL 0x3f88af2167fc491cL
 0x187348bd88683470L 0xabf6af3c0644d361L 0x845bd6b8f0f9932cL
 0x5c1d477fa42f5cb1L 0xcd70c41890bec610L 0xdfd3402c1fa28aa3L
 0xf0b5cec65606a85eL 0x5f8bf5b5d39c190dL 0xf61254a564308d5L
 0xfd5a030d89f5a808L 0

In [6]:
burst_singlekey_test(size = 16)
#burst_singlekey_test(size = 16)
#burst_singlekey_test(size = 16)
#burst_singlekey_test(size = 16)
#burst_singlekey_test(size = 16)

[0x0L 0x0L 0x2500000004ffL 0x4a040000100281aeL 0xe4d3381a9d82c780L
 0x54cabc5c0f6b0dbaL 0x81d46f2828dd27f2L 0xa8f3fe407ad2d55cL
 0x43a7b42fa4fc3a86L 0x761c1052d546c9e6L 0x48a45eeffe79505cL
 0x645958fbd8b9f6f4L 0xc18f5e9c554bf786L 0x8b845f7ed0838d19L
 0x54776daeb19fa053L 0x3b28c1644a269080L 0xd5eadcb3e772e7b4L
 0x95672850a58fd5d9L 0x7336e64ff84cab2eL 0xa6076ea14ce3afd8L
 0x9ad95c88db5c02f7L 0x8a1d926cc1acb6a6L 0xcc5a20b975a34ff4L
 0x1e9f5c96625a8a49L 0xa144cf4f4d4ed692L 0x2021d6f89dce4143L
 0x81f111c0e38bf117L 0x682e26153377e26cL 0x27f4a993ec9326c1L
 0x7b15e47a9eadfdfdL 0x5960947ebd83c143L 0x121b66f4b7f15caL
 0x10c1c66a7a81575cL 0x36cd89984c5d2619L 0x3c8d5307ccb41010L
 0xde4bdb405de416a1L 0x7e6e8fb5730bd973L 0x36c68520f7dc5b46L
 0xe4a164ab36114298L 0xcfd77393604f64cfL] [0x270002002601abL 0x0L]


In [32]:
    
import time
def search_key_test(searchspace = 16):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = SEARCH_KEY
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ searchspace << 24 ^ packetid << 48
    databuff = []
    #cryptbuff = []
    databuff.append(head)
    key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
    #keybuffer = np.getbuffer(key)
    key[0] = 0xFFFFFFFFFFFFFF00
    print '#', key
    data = np.array([randrange(2**base) for i in range (datasize/base)], dtype=np.uint64)
    #databuffer = np.getbuffer(data)

    mode = AES.MODE_ECB
    aes = AES.new(key, mode)
    encryptedbuffer = aes.encrypt(data)
    encrypted = np.frombuffer(encryptedbuffer, dtype=np.uint64)
        
    key[0] = key[0] & 0xFFFFFFFFFFF00000
    for i in key:
        databuff.append(i)
    for i in data:
        databuff.append(i)
    for i in encrypted:
        databuff.append(i)
            
    command, data = encapsulate_packet(0, 2, databuff)
    data = np.array(data, dtype=np.uint64)
    command = np.array([command, 0], dtype=np.uint64)
    #cryptbuff = np.array(cryptbuff, dtype=np.uint64)
    print data, command
    task_writeDMA(data)
    task_writeDMA(command)
    task_writeDMA(null)
    fifo_status = np.array([0, 0x00000000000000AB], dtype=np.uint64)
    task_writeDMA(fifo_status)
    #time.sleep(1)
    for i in range(100000):
        header = task_readDIO(3, 1)
        if int(header[0]) & 0xFF == 0xea:
            if packetid == int(header[0]) >> 48:
                print "packet recieved"
                cryptbuff1 = task_readDMA(13)
                print hex(cryptbuff1[5] - 0x1e)
                print cryptbuff1 
                print 'Count #', i
                break
                #if (cryptbuff1[4:] == cryptbuff).all():
                #    print "encryption is correct"
            #else:
            #    print "Packet not recieved"

NameError: name 'searchspace' is not defined

In [6]:
%time search_key_test(searchspace = 0)

# [0xffffffffffffff00L 0xc4647e18e61da4aaL 0xe00f1e2b5d60db62L
 0x2c89baff4dc5513cL]
[0x0L 0x0L 0x900000002ffL 0x2d510000000381aeL 0xfffffffffff00000L
 0xc4647e18e61da4aaL 0xe00f1e2b5d60db62L 0x2c89baff4dc5513cL
 0x21b92bc6fde680c6L 0xcf90c30d547105bbL 0x46d1f43433defb2bL
 0xda1a799f395b0d05L] [0xb0002000a01abL 0x0L]
packet recieved
0xfff00L
[0xabL 0x0L 0xa00020000ffL 0x2d510000000381eaL 0xfff1eL 0xfff1eL 0xfff1eL
 0xfff1eL 0xfff1eL 0xfff1eL 0xfff1eL 0xfff1eL 0xfff1eL]
Count # 1
CPU times: user 0 ns, sys: 143 ms, total: 143 ms
Wall time: 162 ms


In [13]:
fifo_status = np.array([0, 0x00000000000000AB], dtype=np.uint64)
task_writeDMA(fifo_status)
print task_readDMA(32)

[0xabL 0x0L 0x2100010000ffL 0xcf430000100281eaL 0xceec0dbeedf5b883L
 0xbd47eb22ca2f2a7cL 0xe2ebe170a6634263L 0x6d21f2029468ad80L
 0x5b3429261c3ae83cL 0x698647ec76d46531L 0xc6f3d3df5475ba23L
 0x11bb09c022fbb886L 0x5f0ba1d471052646L 0xf36d5163c8baad4dL
 0xb14a9235e76ef530L 0xdf16482ef34e7e36L 0xb7afe5ab81d36340L
 0xeb22e7f2f858617dL 0x1e30ee7e017121cfL 0x6702d8521a58ba05L
 0x2d065c45d6c2cd5eL 0x39ebdf9cb6d6bcc6L 0xb9e79b3542cc16c5L
 0xc18c68f24853f26L 0x7eeb8a2ee7d72cf5L 0xa7f41885c2908bd8L
 0x6b8b2ee314fd018L 0x30c2944974dc8e8aL 0xf2cca5bd1aabab14L
 0x7fd95e9ee2b1758cL 0x2bbd28c42ef9ce54L 0x90573992e7cc1b3cL]


In [22]:
burst_pairs_test(size = 16)
burst_singlekey_test(size = 16)
burst_pairs_test(size = 16)
burst_singlekey_test(size = 16)
burst_pairs_test(size = 16)
burst_singlekey_test(size = 16)
burst_pairs_test(size = 16)
burst_singlekey_test(size = 16)

packet recieved
encryption is incorrect
wrong/no packet recieved
wrong/no packet recieved
wrong/no packet recieved
